<a href="https://colab.research.google.com/github/t-sasatani/wpt-analysis/blob/main/examples/measurement_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !pip install git+https://github.com/t-sasatani/wpt-tools

In [ ]:
from wpt_tools.analysis import RichNetwork, nw_tools

### Create instance of analysis and set initial variables

In [ ]:
example_nw = RichNetwork.from_touchstone('./assets/sample.s2p')
example_nw.set_f_target_range(target_f=6.78e6, range_f=1e6)


### Plot Z-parameters (full-range)

In [ ]:
nw_tools.plot_z_full(example_nw)

### Efficiency and optimal load analysis (for general 2-port networks)
This becomes unstable when the analysis frequency is far from the resonant frequency.
This is probably because to S to Z conversion becomes unstable.

In [ ]:
f_opt, eff_opt, r_opt, x_opt = nw_tools.analyze_efficiency(
    rich_nw=example_nw,
    show_plot=1,
    rx_port=1
    )

### Curve-fitting and Z-matrix plot (only for 2-port measurement of Series-Series LCR coupling)

In [ ]:
# For maximum efficiency analysis
target_f = 6.78e6
range_f = 1e6

ls1, cs1, rs1, ls2, cs2, rs2, lm = nw_tools.fit_z_narrow(
    rich_nw=example_nw,
    show_plot=1,
    )

### Optimal load visualization

In [ ]:
nw_tools.plot_optimal_load(
    rich_nw=example_nw,
    min_rez=0.1,
    max_rez=50,
    step_rez=0.2,
    min_imz=-200,
    max_imz=200,
    step_imz=1,
    input_voltage=5,
    rx_port=1
)

### Capacitor filter calculator
![](https://github.com/t-sasatani/wpt-analysis/blob/main/examples/img/CpCsRltopology.png?raw=1)

In [ ]:
nw_tools.calc_rxc_filter(
    rich_nw=example_nw,
    rx_port = 1,
    rload = 100,
    c_network = 'CpCsRl'
)